# Visualization: Trading Session

In [48]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [49]:
from trading_bot.agent import Agent

model_name = 'FPT_tdqn_1'
test_stock = 'data/FPT20232024.csv'
window_size = 10
debug = True

agent = Agent(window_size + 1, pretrained=True, model_name=model_name)

### 2. Load test data

In [51]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Price']]
# rename feature column names
df = df.rename(columns={'Price': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2023-01-03,80000
1,2023-01-04,80000
2,2023-01-05,80600
3,2023-01-06,80300
4,2023-01-09,80300


### 3. Running Eval

In [52]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

# Set logging level to DEBUG
logging.basicConfig(level=logging.DEBUG)

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result)


2024-04-12 17:03:17 DESKTOP-RSIH8GO root[2792] DEBUG switching to TensorFlow for CPU
2024-04-12 17:03:17 DESKTOP-RSIH8GO root[2792] DEBUG Buy 900 shares at: 80000.00VND
2024-04-12 17:03:18 DESKTOP-RSIH8GO root[2792] DEBUG Sell 900 shares at: 80000.00VND | Position: +0.00VND
2024-04-12 17:03:18 DESKTOP-RSIH8GO root[2792] DEBUG Buy 900 shares at: 80300.00VND
2024-04-12 17:03:18 DESKTOP-RSIH8GO root[2792] DEBUG Sell 900 shares at: 80300.00VND | Position: +0.00VND
2024-04-12 17:03:18 DESKTOP-RSIH8GO root[2792] DEBUG Buy 300 shares at: 80700.00VND
2024-04-12 17:03:18 DESKTOP-RSIH8GO root[2792] DEBUG Sell 300 shares at: 80000.00VND | Position: -210000.00VND
2024-04-12 17:03:18 DESKTOP-RSIH8GO root[2792] DEBUG Buy 300 shares at: 82000.00VND
2024-04-12 17:03:18 DESKTOP-RSIH8GO root[2792] DEBUG Buy 300 shares at: 81900.00VND
2024-04-12 17:03:18 DESKTOP-RSIH8GO root[2792] DEBUG Buy 300 shares at: 80100.00VND
2024-04-12 17:03:18 DESKTOP-RSIH8GO root[2792] DEBUG Sell 900 shares at: 80700.00VND | P

### 4. Visualize

In [53]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='.2f', title='Price (VND)'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='.2', title='Price (VND)'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [55]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

In [57]:
df.head()

,date,actual,position,action
0,2023-01-03,80000,80000,HOLD
1,2023-01-04,80000,80000,Buy
2,2023-01-05,80600,80000,SELL
3,2023-01-06,80300,80600,HOLD
4,2023-01-09,80300,80300,Buy


In [58]:
import pandas as pd
import os

# Đường dẫn đến thư mục
directory = r'D:\Test1\trading-bot-master\extra'

# Đảm bảo rằng đường dẫn tồn tại
if not os.path.exists(directory):
    os.makedirs(directory)

# Lưu DataFrame thành file CSV
df.to_csv(os.path.join(directory, f'{model_name}_result.csv'), index=False)
